In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [16]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}

In [17]:
df = pd.DataFrame()

#sayfa sayısı kadar döndürür (4.sayfaya kadar döndürdü)
for pageNumber in range(1, 4):
    r = requests.get(f"https://www.kariyer.net/is-ilanlari/bilisim-{pageNumber}", headers=header)
    soup = BeautifulSoup(r.content, "lxml")
    jobNotices  = soup.find_all("div", attrs={"class":"list-items"})

    for notice in jobNotices:

        # Sayfadan ilan linkleri oluşturma
        url = notice.a.get("jobid")

        link = f"https://api.kariyer.net/job?jobId={url}&memberId=30529505"

        response = requests.get(link, headers=header)

        data = response.json()

        # Json dosyasından sayfa kriterlerini çekiyoruz
        title = data["data"]["jobGeneralInformation"]["title"]
        qualifications = data["data"]["jobGeneralInformation"]["qualifications"]
        location = data["data"]["jobGeneralInformation"]["locationText"]
        totalApplication = data["data"]["jobIstatistics"]["totalApplication"]
        workType = data["data"]["jobPositionInformation"]["workTypeText"]
        positionName = data["data"]["jobPositionInformation"]["positionName"]
        positionLevel = data["data"]["jobPositionInformation"]["positionLevelText"]
        positionDepartment = data["data"]["jobPositionInformation"]["workAreas"][0]["name"]
        jobCandidateCriteria =  data["data"]["jobCandidateCriteria"]["educationLevelText"] #liste
        experience =  data["data"]["jobCandidateCriteria"]["experienceText"]
        companyInfo = data["data"]["jobCompanyInformation"]["companyInfo"].strip()


        #Her bir ilan için sözlük oluşturulur.
        job_dict = {"Başlık": title, 
        "İş Tanımı": qualifications, 
        "Şehir": location, 
        "Başvuru Sayısı": totalApplication, 
        "Çalışma Şekli" :  workType, 
        "Pozisyon İsmi": positionName, 
        "Pozisyon Seviyesi": positionLevel,
        "Departman": positionDepartment,
        "Kriterler" : jobCandidateCriteria,
        "Tecrube" : experience,
        "Şirket Tanımı" : companyInfo
        }

        #Oluşturulan sözlükler df ile birleştirilir.
        df_dictionary = pd.DataFrame([job_dict])
        df = pd.concat([df, df_dictionary], ignore_index=True)

        


In [18]:
df.head(5)

,Başlık,İş Tanımı,Şehir,Başvuru Sayısı,Çalışma Şekli,Pozisyon İsmi,Pozisyon Seviyesi,Departman,Kriterler,Tecrube,Şirket Tanımı
0,Yazılım Mühendisi,"Danışmanlığını yürüttüğümüz, İmalat sektöründe...",Bursa,93,Tam Zamanlı,Uzman Yazılım Mühendisi,Uzman,Bilgi Teknolojileri / IT,"[Üniversite(Mezun), Yüksek Lisans(Öğrenci), Yü...",3 - 8 yıl arası,Kariyer.net olarak 21 yıldır dijital teknoloji...
1,Kurumsal Satış ve Pazarlama Sorumlusu,Genel Nitelikler;Lise veya tercihen üniversite...,İstanbul (Tümü),169,Tam Zamanlı,Kurumsal Satış Sorumlusu,Uzman,Pazarlama,"[Lise(Mezun), Ön Lisans(Mezun), Üniversite(Mez...",En az 4 yıl tecrübeli,2001 yılında kurulan firmamız; \r\nDünyaca tan...
2,Yurtdışı Satış Sorumlusu,"Isıtma, soğutma ve enerji sektörüne yönelik ek...",İstanbul(Asya),84,Tam Zamanlı,Yurtdışı Satış Sorumlusu,None,Satış ve Pazarlama,"[Üniversite(Mezun), Ön Lisans(Mezun)]",En az 2 yıl tecrübeli,
3,Mağaza Satış Danışmanı Reeder Torium Avm,GENEL NİTELİKLER VE İŞ TANIMI İstanbul Tori...,İstanbul(Avr.),392,Tam Zamanlı,Mağaza Satış Danışmanı,None,Satış ve Pazarlama,"[Lise(Mezun), Ön Lisans(Mezun), Üniversite(Mez...",En az 1 yıl tecrübeli,
4,Fullstack .Net Developer,Job DescriptionSoft & Solution is a leading co...,İstanbul(Avr.),148,Full-Time,Software Development Süpervisor,Manager candidate,Information Technology,[Bachelor’s(Graduate)],2 - 5 years of experience,As a leading company since 2011 in the field o...


In [12]:
df.shape

(156, 11)

In [14]:
file_name = 'kariyer.xlsx'

df.to_excel(file_name)